## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-15 20:04:20.891058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 20:04:21.749554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "scalar1"

## Loading data

See `cnn/data.py`.

In [6]:
from cnn_data import load_data
X_train, Y_train, X_test, Y_test = load_data(task_name)

load_data from datafile /data/wifeng/photon-jet/data/h5/pi0_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/gamma_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/scalar1_40-250GeV_100k.h5
rng=Generator(PCG64), perm=[245755 266561 162237 ... 223106 154045 228914], n_train=21000, n_test=27900


In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (21000, 4, 16, 1) (21000, 4, 128, 1)
  labels: (21000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-15 20:04:27.369154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:88:00.0, compute capability: 7.5
2024-01-15 20:04:27.370193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-01-15 20:04:27.371026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5
2024-01-15 20:04:27.371738: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9804 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
merge = concatenate([flat1, flat2])

In [12]:
# interpretation model
hidden1 = Dense(20, activation='relu')(merge)
hidden2 = Dense(20, activation='relu')(hidden1)
output = Dense(3)(hidden2)
cnn = Model(inputs=[visible1, visible2], outputs=output)

In [13]:
# summarize layers
# print(cnn.summary())

In [14]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>]

## Train the model

In [22]:
def train_iteration(lr, epochs):
    print(f"=== Training with lr={lr} for {epochs} epochs ===")
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=128,
        validation_data=(X_test, Y_test),
    )

In [23]:
train_iteration(lr=1e-2, epochs=5)

=== Training with lr=0.01 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 4s 12ms/step - loss: 0.0868 - accuracy: 0.9767 - val_loss: 0.1339 - val_accuracy: 0.9716
Epoch 2/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0646 - accuracy: 0.9809 - val_loss: 1.2187 - val_accuracy: 0.8557
Epoch 3/5
165/165 [==============================] - 2s 9ms/step - loss: 0.2955 - accuracy: 0.9123 - val_loss: 0.1240 - val_accuracy: 0.9586
Epoch 4/5
165/165 [==============================] - 2s 9ms/step - loss: 0.1110 - accuracy: 0.9630 - val_loss: 0.1056 - val_accuracy: 0.9664
Epoch 5/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0991 - accuracy: 0.9689 - val_loss: 0.0959 - val_accuracy: 0.9716


In [24]:
train_iteration(lr=2e-3, epochs=5)

=== Training with lr=0.002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 3s 10ms/step - loss: 0.0795 - accuracy: 0.9747 - val_loss: 0.0879 - val_accuracy: 0.9753
Epoch 2/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0718 - accuracy: 0.9778 - val_loss: 0.0886 - val_accuracy: 0.9744
Epoch 3/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0706 - accuracy: 0.9785 - val_loss: 0.0974 - val_accuracy: 0.9721
Epoch 4/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0672 - accuracy: 0.9790 - val_loss: 0.0932 - val_accuracy: 0.9766
Epoch 5/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0656 - accuracy: 0.9800 - val_loss: 0.0927 - val_accuracy: 0.9761


In [25]:
train_iteration(lr=1e-3, epochs=5)

=== Training with lr=0.001 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 3s 9ms/step - loss: 0.0562 - accuracy: 0.9827 - val_loss: 0.0919 - val_accuracy: 0.9770
Epoch 2/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0508 - accuracy: 0.9844 - val_loss: 0.0943 - val_accuracy: 0.9794
Epoch 3/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0503 - accuracy: 0.9852 - val_loss: 0.0937 - val_accuracy: 0.9788
Epoch 4/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0484 - accuracy: 0.9852 - val_loss: 0.0949 - val_accuracy: 0.9793
Epoch 5/5
165/165 [==============================] - 1s 5ms/step - loss: 0.0459 - accuracy: 0.9861 - val_loss: 0.0960 - val_accuracy: 0.9798


In [26]:
train_iteration(lr=2e-4, epochs=5)

=== Training with lr=0.0002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 3s 9ms/step - loss: 0.0413 - accuracy: 0.9876 - val_loss: 0.0934 - val_accuracy: 0.9801
Epoch 2/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0402 - accuracy: 0.9879 - val_loss: 0.0949 - val_accuracy: 0.9808
Epoch 3/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0399 - accuracy: 0.9883 - val_loss: 0.0963 - val_accuracy: 0.9801
Epoch 4/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0391 - accuracy: 0.9880 - val_loss: 0.0986 - val_accuracy: 0.9803
Epoch 5/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0382 - accuracy: 0.9884 - val_loss: 0.0962 - val_accuracy: 0.9803


In [27]:
train_iteration(lr=2e-5, epochs=30)

=== Training with lr=2e-05 for 30 epochs ===
Epoch 1/30
165/165 [==============================] - 3s 9ms/step - loss: 0.0373 - accuracy: 0.9890 - val_loss: 0.0971 - val_accuracy: 0.9805
Epoch 2/30
165/165 [==============================] - 1s 6ms/step - loss: 0.0370 - accuracy: 0.9891 - val_loss: 0.0976 - val_accuracy: 0.9806
Epoch 3/30
165/165 [==============================] - 1s 7ms/step - loss: 0.0368 - accuracy: 0.9890 - val_loss: 0.0975 - val_accuracy: 0.9801
Epoch 4/30
165/165 [==============================] - 1s 6ms/step - loss: 0.0368 - accuracy: 0.9892 - val_loss: 0.0978 - val_accuracy: 0.9802
Epoch 5/30
165/165 [==============================] - 1s 6ms/step - loss: 0.0367 - accuracy: 0.9890 - val_loss: 0.0979 - val_accuracy: 0.9803
Epoch 6/30
165/165 [==============================] - 1s 7ms/step - loss: 0.0366 - accuracy: 0.9891 - val_loss: 0.0984 - val_accuracy: 0.9802
Epoch 7/30
165/165 [==============================] - 1s 7ms/step - loss: 0.0366 - accuracy: 0.9892 - v

In [28]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

task name: scalar1
INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v3.0/scalar1_cnn/assets


INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v3.0/scalar1_cnn/assets


In [ ]:
import json

os.makedirs(f"{output_dir}/cnn", exist_ok=True)
with open(f"{output_dir}/cnn/{task_name}_train_history.json", "w") as fout:
    json.dump({
        key: history.history[key] for key in ["loss", "val_loss", "acc", "val_acc"]
    }, fout)